# Homework 14 
## Mapping Part 2 : Group 1


### Kate Cough
### Due: July 15 2017


In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pysal as ps


%matplotlib inline

In [2]:
powerplants = pd.read_csv('/Users/kaitlincough/Documents/Lede/foundation/14-homework-powerplants/powerplants.csv')
powerplants.head(2)

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,Zip,Street_Add,PrimSource,...,Solar_MW,Wind_MW,Geo_MW,Other_MW,source_des,tech_desc,Source,Period,Latitude,Longitude
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,35476,19001 Lock 17 Road,hydroelectric,...,0.0,0.0,NaN,NaN,Hydroelectric = 56 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",201607,33.458665,-87.356823
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,36512,North Highway 43,coal,...,0.0,0.0,NaN,NaN,"Coal = 1088.5 MW, Natural Gas = 1072 MW",Conventional Steam Coal; Natural Gas Fired Com...,"EIA-860, EIA-860M and EIA-923",201607,31.006900,-88.010300


In [ ]:
#import the files we need and look at them. just powerplants for now.
powerplants = pd.read_csv('/Users/kaitlincough/Documents/Lede/foundation/14-homework-powerplants/powerplants.csv')
#apply the function to each row of powerplants to turn it into a geodatafram
points = powerplants.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)
#actually make it a geodataframe
powerplants = gpd.GeoDataFrame(powerplants, geometry=points)
#set the crs so we're not sad later
powerplants.crs = {'init':'epsg:5070'}
powerplants.head(2)

In [ ]:
#always a good idea to plot it, just to see right. 
#and to make it not totally awful so our eyes don't burn
#alpha is for opacity, markeredgewidth and sizes are just 
#to change the dot sizes
#keep the axis for now so we can see the lat/long

powerplants.plot(figsize=(20,10), color='#3498DB', markeredgewidth=0, markersize=5, alpha=0.5)


Lesson: there are a LOT of powerplants in the US. 

In [ ]:
#let's plot it without the axis and set the x,y limits
#so we can just focus on the continental us
ax = powerplants.plot(figsize=(20,10), color='#3498DB', markeredgewidth=0, markersize=5, alpha=0.5)
# ax.axis('off')

#set the limits so it stays within these lat/long.
#we know the limits we want to use by looking at the map above
ax.set_xlim([-130,-60])
ax.set_ylim([25,50])

In [ ]:
#import the shape file. remember it's reading EVERYTHING in the folder, not just the .shp
#but we only import the .shp

states = gpd.read_file('/Users/kaitlincough/Documents/Lede/foundation/14-homework-powerplants/cb_2016_us_state_500k/cb_2016_us_state_500k.shp')
#change the crs to match the one above, use the conus albers one
#which is good for the united states but sad for other places
states.crs = {'init':'epsg:5070'}
states.head(2)

In [ ]:
#plot the states, turn off the axis, make the colors not horrendous

ax = states.plot(figsize=(20,10), color='lightgrey', edgecolor='white')
# ax.axis('off')

In [ ]:
ax = states.plot(figsize=(20,10), color='lightgrey', edgecolor='white')
powerplants.plot(figsize=(20,10), ax=ax, color='#3498DB', markeredgewidth=0, markersize=5, alpha=0.5)
ax.set_xlim([-130,-60])
ax.set_ylim([25,50])
# ax.axis('off')

### 1. Make a map of power plants in the US, color coded by type


In [ ]:
#first let's remember what's in there:
powerplants.head(1)

In [ ]:
#use a nice color map: https://matplotlib.org/users/colormaps.html
#have it pay attention to the column PrimSource

ax = powerplants.plot(column='PrimSource', cmap='GnBu', figsize=(20,20), markersize=5, alpha=0.5)
# ax.axis('off')

In [ ]:
#we can make this look nicer by plotting it on top
#of the state's geometry

ax = states.plot(figsize=(20,10), color='lightgrey', edgecolor='white')
powerplants.plot(ax=ax, column='PrimSource', cmap='GnBu', figsize=(20,20), markersize=5, alpha=0.5)

ax.set_xlim([-130,-60])
ax.set_ylim([25,50])
# ax.axis('off')

### 2. Make a map of air quality stations, color coded by average air quality


In [ ]:
#we need this file with the air quality monitoring info

airqual = pd.read_csv('/Users/kaitlincough/Documents/Lede/foundation/14-homework-powerplants/annual_conc_by_monitor_2017.csv')
airqual.head(2)

In [ ]:
#turn the airqual into a geodataframe 
#read in only useful columns

airqual = pd.read_csv('/Users/kaitlincough/Documents/Lede/foundation/14-homework-powerplants/pm25_annual_fixed.csv', usecols = ['Latitude', 'Longitude', 'Datum', 'Arithmetic Mean', '1st Max Value', 'State Name', 'County Name', 'City Name'])
points = airqual.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)
airqual = gpd.GeoDataFrame(airqual, geometry=points)
airqual.crs = {'init':'epsg:5070'}
airqual.head()

In [ ]:
ax = states.plot(figsize=(20,10), color='lightgrey', edgecolor='white')

airqual.plot(ax=ax, column='Arithmetic Mean', cmap='Blues', figsize=(20,10))
#focus on the continental US
ax.set_xlim([-130,-60])
ax.set_ylim([25,50])

### 3. Make a map of air quality stations, color coded by worst air quality ever 


In [ ]:
ax = states.plot(figsize=(20,10), color='lightgrey', edgecolor='white')


airqual.plot(ax=ax, column='1st Max Value', cmap='Greens')

ax.set_xlim([-130,-60])
ax.set_ylim([25,50])

### 4. Map the power plants owned by the top 5 most common energy companies as red. Map the rest of the plants as dark grey.

In [ ]:
powerplants.head(2)

In [ ]:
#here's the series with the parameter
powerplants['Utility_Na'].value_counts().head().index

In [ ]:
#save it in a variable
top_five = powerplants['Utility_Na'].value_counts().head().index
top_five

In [ ]:
#save it to a list
top_list = top_five.tolist()
top_list

In [ ]:
#check to see if a plant is in the list.
#if it is, make it red, if not, grey
powerplants[powerplants['Utility_Na'].isin(top_list)].plot(markersize=3, alpha=0.5, color='red')

In [ ]:
#create a separate data frame plot those in red, and then you plot them on top of one another
ax = states.plot(figsize=(20,20), color = 'lightgray', edgecolor = 'white')
# ax.axis('off')
powerplants[~powerplants['Utility_Na'].isin(top_list)].plot(markersize=3, alpha=0.5, color='darkgrey', ax=ax, figsize=(20,20))
powerplants[powerplants['Utility_Na'].isin(top_list)].plot(markersize=3, alpha=0.5, color='red', ax=ax, figsize=(20,20))

### 5. Map all of the nuclear power plants in red, with the rest of the plants as dark grey.


In [ ]:
powerplants.PrimSource.value_counts()

In [ ]:
nuclear = powerplants[(powerplants['PrimSource'] == 'nuclear')]
nuclear.plot(color='red')


In [ ]:
non_nuclear = powerplants[(powerplants['PrimSource'] != 'nuclear')]
non_nuclear.plot(color='darkgrey')

In [ ]:
ax = states.plot(figsize=(20,20), color = 'lightgray', edgecolor = 'white')
# ax.axis('off')

nuclear.plot(ax=ax, color='red')
non_nuclear.plot(ax=ax, color='darkgrey', alpha=0.5)

ax.set_xlim([-130,-60])
ax.set_ylim([25,50])

### 6. Map power plants that produce energy through both natural gas and oil.

In [ ]:
powerplants['tech_desc'].value_counts()

In [ ]:
#find the powerplants, using string contains,
#that use natural gas or petroleum

gas_oil = powerplants[powerplants['source_des'].str.contains('Natural Gas|Petroleum')]
gas_oil.plot(color='green')

In [ ]:
#plot them on the original map
ax = states.plot(color='lightgrey', edgecolor='white', figsize = (20,20))
# ax.axis('off')

gas_oil.plot(color='green', figsize=(20,20), ax=ax)